<center><h1>Conversion methods</h1></center>

This notebook defines the methods used to convert a [Molecular Design Toolkit](http://moldesign.bionano.autodesk.com) object to JSON.

To "import" these methods into another notebook, run
`%run moldesign_to_json.ipynb`

#### For convenience:
We'll return these objects as a python JSON-like object - you can interact with these as if they were JSON objects in javascript (in particular, you can hit `tab` to see what's in them).

In [8]:
import json, uuid

class PyJSON(dict):
    COUNTER = 0
    def __getattr__(self, key):
        try: return self[key]
        except KeyError: raise AttributeError
    def __setattr__(self, key, val):
        self[key] = val
    def __dir__(self):
        return self.keys()
    def dumps(self):
        return json.dumps(self)
    def display(self):
        """From http://stackoverflow.com/questions/18873066"""
        from IPython.display import display_html, display_javascript
        PyJSON.COUNTER += 1
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.COUNTER),
                     raw=True)
        display_javascript(('require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {'
                            "\ndocument.getElementById('%s').appendChild(renderjson(%s))"
                            "\n});") % (self.COUNTER, self.dumps()), raw=True)

## Drivers

These are the routines you call to convert `moldesign` objects into JSON. Because MDT's design, there's one for single points and one for trajectories, but they both produce the same type of JSON objects.

In [6]:
def convert(mol, provenance=None):
    json = PyJSON(name=mol.name)
    if provenance is not None:
        json.provenance = provenance
    json.topology = create_topology_object(mol)
    json.states = create_states(mol)
    return json

def convert_trajectory(traj, **kwargs):
    json = convert(traj.mol, **kwargs)
    json.states = create_states(traj)
    return json

# Sub-objects
Everything below drives the converter.

First, here are routines to create these top-level objects:
 - `topology`
 - `states`

In [15]:
def create_topology_object(m):
    """Create the top-level topology object"""
    topo = PyJSON()
    topo.atomArray = create_atom_array(m.atoms)
    topo.bondArray = create_bonds(m)
    if m.num_residues > 1:
        topo.groups = create_groups(m)
    topo.charge = m.charge.value_in(u.q_e)
    return topo

In [10]:
def create_states(m):
    """Create a list of states from a molecule or trajectory
    """
    states = []
    if isinstance(m, mdt.Molecule):
         states.append(make_state(m,
                                  positions=m.positions,
                                  momenta=m.momenta,
                                  properties=m.properties,
                                  ))
    else:
        assert isinstance(m, mdt.Trajectory)
        for frame in m.frames:
            states.append(make_state(m,
                                     positions=frame.state,
                                     momenta=frame.momenta,
                                     properties=m.properties))
    return states

In [16]:
def create_quantity(q):
    units = q.units
    q = q.defunits().magnitude
    
    try: q = q.tolist()
    except AttributeError: pass
    
    return PyJSON(val=q,
                  units=str(units))

### Topology components

The routines in this section create various parts of the topology data structure.

In [16]:
def create_groups(m):
    """Create 'groups' (will be residues and chains for biomolecules)"""
    return PyJSON(residueArray=create_residue_array(m.residues),
                   chainArray=create_chain_array(m.chains))

In [13]:
def create_atom_array(atoms):
    """Creates the atomArray JSON object"""
    return PyJSON(names=[atom.name for atom in atoms],
                   atomicNumbers=[atom.atnum for atom in atoms],
                   formalCharges=[atom.formal_charge.value_in(u.q_e) for atom in atoms],
                   sequenceNumbers=[atom.pdbindex for atom in atoms],
                   masses=create_quantity(u.array([atom.mass for atom in atoms]))
                  )

In [26]:
def create_bonds(m):
    """Creates the bondArray JSON object"""
    return PyJSON(atomIndices=[(b.a1.index, b.a2.index) for b in m.bonds],
                   lewisOrders=[b.order for b in m.bonds],
                   wibergOrders=[None for b in m.bonds])

In [27]:
def create_residue_array(residues):
    """Creates the residueArray JSON object"""
    return PyJSON(residueCodes=[r.resname for r in residues],  # 3 letter residue codes
                   sequenceNumbers=[r.pdbindex for r in residues],
                   atomIndices=[[atom for atom in r.atoms] for r in residues])

In [28]:
def create_chain_array(chains):
    """Creates the chainArray JSON object"""
    return PyJSON(chainNames=[c.name for c in chains],
                   residueIndices=[[residue.index for residue in c.residues] for c in chains])

### State components

Routines here create parts of the `state` data structures.

In [ ]:
def make_state(mol, positions=None, momenta=None, properties=None, time=None, step=None, description=None):
    """Creates a dynamical state object"""
    state = PyJSON()
    if positions is not None:
        state.positions = create_quantity(positions)
    if momenta is not None:
        state.momenta = create_quantity(momenta)
    if time is not None:
        state.time = create_quantity(time)
    if step is not None:
        state.step = step
    if description is not None:
        state.description = description
    if properties:
        state.calculated = create_properties(mol, properties)
    return state

In [9]:
def create_properties(mol, properties):
    """Create a `properties` object that stores calculated properties"""
    props = PyJSON()
    props.energy_model = mol.energy_model.__class__.__name__
    props.parameters = PyJSON(mol.energy_model.params)
    props.update(properties)
    return props